In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import imageio

import tensorflow as tf
from tensorflow.keras import layers, models
from keras.layers import Dense, Dropout, Activation

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data/data.csv')
data = data[data.columns[1:]].set_index('date')
original_data = data

code = pd.read_csv('data/kospi200code.csv')
code = code['code']

In [ ]:
data_code = [] # 3년 이상 존재하는 코드
for i in range(len(code)):
    data = original_data
    data = original_data[original_data['code']==code[i]].reset_index().sort_values(by=['date'])['close']
    if data.shape[0]>900:
        data_code.append(i)
    else : 
        continue

#### 전체 만들기

In [ ]:
data = original_data
data = original_data[original_data['code']==code[5]].reset_index().sort_values(by=['date'])['close']

In [ ]:
data.plot()

In [ ]:
data = original_data
data = original_data[original_data['code']==code[5]].reset_index().sort_values(by=['date'])['close']

df_train = data.iloc[:int(len(data)*0.8),]
df_test = data.iloc[int(len(data)*0.8):,]
mean = df_train.mean(axis=0)
std = df_train.std(axis=0)
df_train -= mean
df_train /= std
df_test -= mean
df_test /= std

# for j in range(data.shape[0]-5):
#     plt.style.use('dark_background')
#     plt.figure(figsize=(4,4),dpi=10)
#     data[j:j+5].plot()
#     plt.axis('off')
#     fig = plt.gcf()
#     fig.savefig('data/cnn_test/{}.png'.format(j), dpi=fig.dpi)

---------------------

--------------------

In [ ]:
img_vec=[]
for j in range(len(data)-5):
    img = imageio.imread('data/cnn/1-{}.png'.format(j), pilmode='RGB')
    img = img/255
    img_vec.append(img)

In [ ]:
len(img_vec)

In [ ]:
y = []
for j in range(len(data)-5):
    if data.iloc[j+4] > data.iloc[j+5]:
        y.append(1)
    else : 
        y.append(0)

In [ ]:
len(y)

In [ ]:
X = img_vec

In [ ]:
train_X = np.array(X[:int(len(data)*0.8)])
test_X = np.array(X[int(len(data)*0.8):])

train_y = np.array(y[:int(len(data)*0.8)])
test_y = np.array(y[int(len(data)*0.8):])

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(64, (1, 1), activation='relu', input_shape=(40, 40, 3)))
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(32, (1, 1), activation='relu'))
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(train_X, train_y, batch_size=4, epochs=50)

In [ ]:
pred = model.predict(test_X)

In [ ]:
predict = []
for i in range(pred.shape[0]):
    predicted=pred[i].argmax()
    predict.append(predicted)

In [ ]:
print('accuracy = ',round((predict == test_y).sum() / len(predict),3)*100,'%')